***Participantes (RM - NOME):***<br>
339624 - Camila<br>
339656 - Cleiton<br>
340214 - Henrique<br>
339708 - Roberto<br>
340192 - Sergio<br>

## **Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

# Bibliotecas utilizadas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import wordcloud as wd
import re
from collections import Counter

import spacy

import nltk
from nltk.stem.rslp import RSLPStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.utils import plot_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pydot


In [ ]:
sb.set_theme(context='notebook', style='whitegrid')

nlp = spacy.load('pt_core_news_md')
rslp = RSLPStemmer()

In [ ]:
print(tf.__version__)
#print(keras.__version__)
print(tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Funções externas

In [ ]:
# função de lematização completa do documento
def stemmer_text(frase):
  tokens = [rslp.stem(w) for w in frase.split()]
  return " ".join(tokens)

# função de lematização completa do documento
def lemmatizer_text(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ for w in doc]
  return " ".join(tokens)

# função de lematização para os verbos do documento
def lemmatizer_verbs(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ if w.pos_ == 'VERB' else w.text for w in doc]
  return " ".join(tokens)

# função de lematização apenas ADV e ADJ
def lemmatizer_adv_adj(frase):
  doc = nlp(frase)
  tokens = [w.lemma_ for w in doc if (w.pos_ == 'ADV' or w.pos_ == 'ADJ')]
  return " ".join(tokens)  

def plural_singular(palavra):
  r2 = r"ses$|zes$|res$"
  r3 = r"ões$|ães$"
  r4 = r"ais$|éis$|óis$|uis$"
  r5 = r"is$"
  r6 = r"eis$"
  if palavra.endswith('s'):
    if re.findall(r2, palavra):
      return palavra[:-2]
    if re.findall(r3, palavra):
      return palavra[:-3] + "ão"
    if re.findall(r4, palavra):
      return palavra[:-2] + "l"
    if re.findall(r5, palavra):
      return palavra[:-1] + "l"
    if re.findall(r6, palavra):
      return palavra[:-1] + "il"
    if palavra.endswith('ns'):
      return palavra[:-2] + "m"
    return palavra[:-1]
  return palavra  

def pre(frase):
  regex = r"[`,.?:;!&\"]"
  palavras = frase.split()
  palavras = [p.lower() for p in palavras if p not in stops]
  palavras = [re.sub(regex, "", p) for p in palavras]
  palavras = [p for p in palavras if len(p) >= 3]
  palavras = [p for p in palavras if not p.isnumeric()]
  palavras = [plural_singular(p) for p in palavras]

  return " ".join(palavras)

In [ ]:
t = "Este fime é muito bom e especial"

d = nlp(t)
for w in d:
    print(w.text, w.pos_, w.lemma_, w.dep_)

# Carregando os Dados

In [ ]:
df = pd.read_csv('./data/reviews-pt-br.csv', encoding='utf-8')

In [ ]:
df.info()

In [ ]:
df.head(10)

## Conferindo se temos dados nulos ou duplicados

In [ ]:
df.isna().sum()

In [ ]:
df[df.duplicated()].count()

## Distribuição das respostas

In [ ]:
df.groupby('sentimento').count()

In [ ]:
# em percentual
round(df.groupby('sentimento').count().texto / df.shape[0] * 100, 2)

In [ ]:
sb.countplot(x=df.sentimento)

## Criando o conjunto de stopwords (NLTK + SPACY)

In [ ]:
stopwords_nltk = nltk.corpus.stopwords.words('portuguese')
stopwords_spacy = nlp.Defaults.stop_words
stops = list(set(stopwords_spacy).union(stopwords_nltk))
print(sorted(stops))

# Analisando o conjunto total das palavras

In [ ]:
# exemplo de críticas no nosso dataframe
print(df.texto[0])
print("-" * 40)
print(df.texto[1536])
print("-" * 40)
print(df.texto[8192])

In [ ]:
# criando um texto completo com todas as críticas
texto_completo = " ".join([texto for texto in df.texto])
texto_completo_pos = " ".join([texto for texto in df[df.sentimento == 'pos'].texto])
texto_completo_neg = " ".join([texto for texto in df[df.sentimento == 'neg'].texto])

In [ ]:
print("QTD de todas as palvaras", len(texto_completo.split()))
print("QTD de todas as palvaras das críticas positivas", len(texto_completo_pos.split()))
print("QTD de todas as palvaras das críticas negativas", len(texto_completo_neg.split()))

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo_pos)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras POSITIVAS da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

In [ ]:
# criando uma wordcloud de todas as palavra sem as stopwords
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False, stopwords = stops).generate(texto_completo_neg)

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras NEGATIVAS da Base de Dados retirando as StopWords", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

## Criando uma função de pré processamento

In [ ]:
palavras = texto_completo.split()
len(palavras)

In [ ]:
# retirando as stopwords das palavras
palavras = [p.lower() for p in palavras if p not in stops]
len(palavras)

In [ ]:
# retirando pontuações das palavras
regex = r"[`,.?:;!&\"]"
palavras = [re.sub(regex, "", p) for p in palavras]
len(palavras)

In [ ]:
#retirando palavras menores que 3 caracteres
palavras = [p for p in palavras if len(p) >= 3]
len(palavras)

In [ ]:
# retirando os números das palavras
palavras = [p for p in palavras if not p.isnumeric()]
len(palavras)

In [ ]:
#convertendo as palavras para o singular
palavras = [plural_singular(p) for p in palavras]
len(palavras)

## Criamos uma função chamada "pre" que iremos usar em nossas análises dos modelos

# Vamos analisar quais palavras são comuns as críticas negativas e positivas

In [ ]:
todas_palavras_pos = texto_completo_pos.split()
todas_palavras_pos = [pre(w) for w in todas_palavras_pos]
todas_palavras_pos = [w for w in todas_palavras_pos if w != '']
ctpp = Counter()
ctpp.update(todas_palavras_pos)
ctpp.most_common(50)

In [ ]:
todas_palavras_neg = texto_completo_neg.split()
todas_palavras_neg = [pre(w) for w in todas_palavras_neg]
todas_palavras_neg = [w for w in todas_palavras_neg if w != '']
ctpn = Counter()
ctpn.update(todas_palavras_neg)
ctpn.most_common(50)

In [ ]:
%%time
p = ctpp.most_common()
n = ctpn.most_common()
r = [[x for x in p if x[0] == y[0]] for y in n]
r = [x for x in r if len(x) != 0]
len(r)

In [ ]:
palavras_comuns = [w[0][0] for w in r]
palavras_comuns[:100]

In [ ]:
# criando uma wordcloud das palavras filtradas
wordcloud = wd.WordCloud(width = 3000, height = 2000, max_words=50, random_state=42, background_color='black', colormap='Blues', collocations=False).fit_words(dict(ctp))

plt.figure(figsize=(20, 10))
plt.title("Todas as palavras da Base de Dados após o Pré Pocessamento", fontdict={'fontsize':24})
plt.imshow(wordcloud) 
plt.axis("off");
plt.show()

## Criando as colunas tratadas

In [ ]:
%%time
#df['stem'] = df.texto.apply(stemmer_text)

In [ ]:
%time
#df['lemm'] = df.texto.apply(lemmatizer_text)

In [ ]:
%%time
#df['verb'] = df.texto.apply(lemmatizer_verbs)

In [ ]:
%%time
#df['pre'] = df.texto.apply(pre)

In [ ]:
#df.to_csv("./data/criticas.csv")
df = pd.read_csv('../criticas.csv')

In [ ]:
df.head(10)

In [ ]:
stops2 = set(stops).union(palavras_comuns[:100])
len(stops2)

# Dividindo a base em treino e teste

In [ ]:
# dividindo com 20% para o treino e random state = 42
df_treino, df_teste = train_test_split(
      df, 
      test_size = 0.2, 
      random_state = 42
  )

In [ ]:
# distribuição das respostas do treino em %
round(df_treino.groupby('sentimento').count() / df.shape[0] * 100, 2).texto

In [ ]:
# distribuição das respostas do teste em %
round(df_teste.groupby('sentimento').count() / df.shape[0] * 100, 2).texto

# Criando o primeiro modelo com árvore de decisão

In [ ]:
%%time

def criar_vetores(base_treino, base_teste, coluna)
    nomes = ['CV ngram(1,1)', 'CV ngram(1,2)', 'CV ngram(1,3)', 'CV ngram(2,2)', 'CV ngram(2,3)', 'CV ngram(3,3)',
                'TF ngram(1,1)', 'TF ngram(1,2)', 'TF ngram(1,3)', 'TF ngram(2,2)', 'TF ngram(2,3)', 'TF ngram(3,3)',]
    vetores[]

    for x in range(1,4):
        for y in range(1,4): 
            if (y < x):
                continue
            vect = CountVectorizer(ngram_range=(x,y), stops_words = stops) 

vect.fit(df_treino['texto'])
vect_treino = vect.transform(df_treino['texto'])
vect_teste = vect.transform(df_teste['texto'])

In [ ]:
%%time
# vetorização em unigramas com CountVectorizer
vect = CountVectorizer(ngram_range=(1,1)) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas com CountVectorizer e sem stopwords
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")


In [ ]:
%%time
# vetorização em unigramas com CountVectorizer e pré processamento
vect = CountVectorizer(ngram_range=(1,1)) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.pre)
vect_treino = vect.transform(df_treino.pre)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.pre)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas com CountVectorizer, sem stopwords e Stemmização
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.stem)
vect_treino = vect.transform(df_treino.stem)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.stem)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas com CountVectorizer, sem stopwords e Lemmatização
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.lemm)
vect_treino = vect.transform(df_treino.lemm)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.lemm)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas com CountVectorizer, sem stopwords e Lemmatização de verbos
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.verb)
vect_treino = vect.transform(df_treino.verb)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.verb)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

## Vamos continuar a analise com a base pre processada e original sem stop words

## Testando o TFIDF

In [ ]:
%%time
# vetorização em unigramas com TfIdf e sem stopwords
vect = TfidfVectorizer(ngram_range=(1,1), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas com TfIdf e pré processamento
vect = TfidfVectorizer(ngram_range=(1,1)) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.pre)
vect_treino = vect.transform(df_treino.pre)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.pre)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

## Observamos um melhor resultado com o CountVectorizer

## Testando modelos com Bigramas

In [ ]:
%%time
# vetorização em unigramas + Bigramas com CountVectorizer e sem stopwords
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em bigramas com CountVectorizer e sem stopwords
vect = CountVectorizer(ngram_range=(2,2), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas + bigramas com CountVectorizer e pré processamento
vect = CountVectorizer(ngram_range=(1,2)) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.pre)
vect_treino = vect.transform(df_treino.pre)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.pre)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em bigramas com CountVectorizer e pré processamento
vect = CountVectorizer(ngram_range=(2,2)) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.pre)
vect_treino = vect.transform(df_treino.pre)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.pre)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

## Testando modelo com Trigramas usando o melhor modelo com Bigramas

In [ ]:
%%time
# vetorização em unigramas + Bigramas + Trigramas com CountVectorizer e sem stopwords
vect = CountVectorizer(ngram_range=(1,3), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

## Como não houve melhora vamos continuar com Unigramas + Bigramas

In [ ]:
%%time
# vetorização em unigramas + Bigramas com CountVectorizer e sem stopwords
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops)#, min_df=10, max_df=0.95) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas + Bigramas com CountVectorizer e sem stopwords
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops, min_df=10) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas + Bigramas com CountVectorizer e sem stopwords
vect = TfidfVectorizer(ngram_range=(1,2), stop_words=stops, min_df=10) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas + Bigramas com CountVectorizer e sem stopwords
vect = HashingVectorizer(ngram_range=(1,2), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

# treinamento do modelo árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

# Random Forest

In [ ]:
%%time
# vetorização em unigramas sem stopwords
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops, min_df=10) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

In [ ]:
%%time
# treinamento do modelo Árvore Aleatória
tree = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas sem stopwords
vect = TfidfVectorizer(ngram_range=(1,2), stop_words=stops, min_df=10) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

In [ ]:
%%time
# treinamento do modelo árvore de decisão
tree = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# vetorização em unigramas sem stopwords
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops, min_df=10) 
# utilizando a base completa sem ajustes
vect.fit(df_treino.texto)
vect_treino = vect.transform(df_treino.texto)

In [ ]:
%%time
# treinamento do modelo árvore de decisão
tree = AdaBoostClassifier(n_estimators=200, random_state=42)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# treinamento do modelo árvore de decisão
tree = BaggingClassifier(n_estimators=200, random_state=42, n_jobs=-1)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

In [ ]:
%%time
# treinamento do modelo árvore de decisão
tree = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
tree.fit(vect_treino, df_treino.sentimento)

# escoragem da classificação na amostra de teste
vect_teste = vect.transform(df_teste.texto)
predito = tree.predict(vect_teste)

# mensuração do resultado pela acurácia e f1 score
accuracy = accuracy_score(df_teste.sentimento, predito)
f1score = f1_score(df_teste.sentimento, predito, average='weighted')

print(f"Acuracidade: {round(accuracy,4)}")
print(f"F1 Score: {round(f1score,4)}")

# MLP

In [ ]:
df.head()

In [ ]:
df_mlp = df[['texto', 'pre']].copy()
df_mlp['target'] = [0 if x == 'pos' else 1 for x in df.sentimento]
df_mlp.head()

In [ ]:
# dividindo com 20% para o treino e random state = 42
df_treino_mlp, df_teste_mlp = train_test_split(
      df_mlp, 
      test_size = 0.2, 
      random_state = 42
  )
df_mlp.info()  

In [ ]:
# distribuição das respostas do treino em %
round(df_treino_mlp.groupby('target').count() / df_mlp.shape[0] * 100, 2).texto

In [ ]:
# distribuição das respostas do teste em %
round(df_teste_mlp.groupby('target').count() / df_mlp.shape[0] * 100, 2).texto

In [ ]:
t = Tokenizer()
t.fit_on_texts(df_mlp.pre)

In [ ]:
t.document_count

In [ ]:
encoded_docs = t.texts_to_matrix(df_mlp.pre, mode='count')
encoded_docs.shape

In [ ]:
# vetorização em unigramas + Bigramas com CountVectorizer e sem stopwords
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops, min_df=50) 
#vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) 
# utilizando a base completa sem ajustes
vect.fit(df_treino_mlp.pre)
vect_treino = vect.transform(df_treino_mlp.pre)

In [ ]:
vect_treino

In [ ]:
X = vect_treino.toarray()
X.shape

In [ ]:
y = df_treino_mlp.target.values.reshape(-1,1)
y.shape

In [ ]:
#Modelo da Rede MLP
model = Sequential()
model.add(Dense(100, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
plot_model(model)

In [ ]:
%%time
#Processo de treinamento com 80% dos dados
history  = model.fit(X, y, epochs=5, validation_split=0.3, batch_size=2)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
vect_teste = vect.transform(df_teste_mlp.pre)
X_teste = vect_teste.toarray()
X_teste.shape

In [ ]:
y_teste = df_teste_mlp.target.values.reshape(-1,1)
y_teste.shape

In [ ]:
#Processo de teste com 30% dos dados que não foram utilizados no treinamento
y_pred_classes  = model.predict_classes(X_teste)
y_pred_classes

In [ ]:
scores = model.evaluate(X_teste, y_teste)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#### **Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.